# DQN for policy II

In [1]:
import numpy as np
import random
from random import sample
import math 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import gc

In [2]:
k = 3 #number of components

In [3]:
def reset():
    st= [0]*(2*k)
    return st

In [41]:
weibull_scale=(14.076,5.934,4.248,8.373,2.046,23.41,4.93,12.148)
weibull_shape=(378.17,108.917,79.65,115.829,170.756,143.747,43.953,278.507)
t_f=(0.012,0.039,0.015,0.036,0.03,0.021,0.018,0.021)
tp=(0.0024,0.032,0.0037,0.0051,0.0074,0.0042,0.0026,0.0052)
time_interval=0.5 #0.03
running_time=137

In [42]:
def env1(action,st,i): 
    reward =0
    
    f = random.weibullvariate(weibull_scale[i],weibull_shape[i])
     
    if action[i] == 0 :
        if f <= st[i]: # fail
            st[i]= 0 #new
            reward= -(time_interval / tp[i])*time_interval * math.ceil(t_f[i]/time_interval)
            st[i+8]=0
        else:
            st[i] += time_interval
            reward = 5
            
    if action == 1 :
        
        st[i]=0
        st[i+8]=0
        reward = -(time_interval / tp[i])*tp[i]
            
           
    return (tuple(st) , reward)



In [43]:
def env(action,st,k): 
    reward = np.zeros(k)
    for i in range(k): 
        if st[i+k]==1: # fail
            st[i]= 0 #new
            reward[i] = -(time_interval / tp[i])*time_interval * math.ceil(t_f[i]/time_interval)
            st[i+k] = 0
        else:
            if action[i] == 1 :        
                st[i] = 0
                st[i+k] = 0
                reward[i] = -(time_interval / tp[i])*tp[i]
            else:
                st[i] += time_interval
                reward[i] = time_interval
    
        f = random.weibullvariate(weibull_scale[i],weibull_shape[i])
        if f <= st[i]: # fail
            st[i+k]= 1
        else:
            st[i+k]= 0

           
    return (st , np.max(reward))

In [44]:
s ,r = env([0,0,0],list((250,2222,2400,0,1,1)),3)
print(s,r)

[250.5, 0, 0, 1, 0, 0] 0.5


In [45]:
import itertools

perms = list(itertools.product([0, 1], repeat=3))
print(perms)
print(perms[4])

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
(1, 0, 0)


In [46]:
indices = [i for i, x in enumerate(perms[3]) if x > 0]
print(indices)

[1, 2]


In [47]:
def action_for_each(index,num_component):
    perms = list(itertools.product([0, 1], repeat=num_component))
    indices = [i for i, x in enumerate(perms[index]) if x > 0]
    return list(perms[index]),indices

In [48]:
action_space_dim = 2*k
state_space_dim = 2**k
gamma = 0.5   # Discount rate
  
#epsilon_decay = 0.995 
learning_rate = 0.01  
learning_rate_decay = 0.01  # 0.01 Learning rate decay
batch_size = 64

In [49]:
def build_model(state_space_dim,action_space_dim):
    model = Sequential()
    model.add(Dense(2, input_dim=state_space_dim, activation='sigmoid'))  
    model.add(Dense(3, activation='sigmoid'))  
    model.add(Dense(action_space_dim, activation='linear'))  
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate, decay=learning_rate_decay))
    return model

In [50]:
model = build_model(2*k,2**k)
#model.predict([[0.2,0.5,0.1]],verbose=0)

C:\Users\ARYA\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [51]:
#state = list((1,1,1,1,1,1,11,1,1,11,1,1,0,1,1,1))
model = build_model(16,2**8)
state = [1,1,1,1,1,1,11,1,1,11,1,1,0,1,1,1]
state[0]
d = model.predict([state],verbose=0)[0][[1,0,0,0,0,0,0,0]]

#d = model.predict([state],verbose=0)[0]
print(d)
d= 5
print(d)

[-0.04859623 -0.11541748 -0.11541748 -0.11541748 -0.11541748 -0.11541748
 -0.11541748 -0.11541748]
5


In [52]:
def choose_action(epsilon, state):
    if (np.random.random() <= epsilon):
        return random.choice([0,1]) 
    else:
        act_values = model.predict([state],verbose=0)
        return np.argmax(act_values)

In [53]:
choose_action(0.52, state)

0

In [54]:
a , _ = action_for_each(1,k)
print(a)

[0, 0, 1]


In [55]:
MEMORY_CAPACITY = 1000
memory =[]

# Define a function to update the replay memory
def update_memory(state, action, reward, next_state):
    memory.append((state, action, reward, next_state))
    if len(memory) > MEMORY_CAPACITY:
        memory.pop(0) #remove and return the first element
    return memory

In [56]:
def replay(memory,batch_size,state,epsilon):
    minibatch = random.sample(memory, min(len(memory), batch_size))
    state_batch, Q_batch = [], []

    for state, action, reward, next_state in minibatch:
        Q_target = model.predict([state],verbose=0)  # Use the model to predict the target
        if j == running_time//time_interval:
            Q_target[0][action] = reward
        else:
            Q_target[0][action] = reward + gamma * np.max(model.predict([next_state],verbose=0)[0])
        
        state_batch.append(state)
        Q_batch.append(Q_target)

    model.fit(np.array(state_batch), np.array(Q_batch), batch_size=len(state_batch), verbose=0)
    #if epsilon > epsilon_min:
        #epsilon *= epsilon_decay
    epsilon = 1 / (episode + 1)
    #return Q_target

In [57]:
running_time//time_interval

274.0

In [58]:
def reset_tensorflow_keras_backend():
    # to be further investigated, but this seems to be enough
    tf.keras.backend.clear_session()
    #tf.reset_default_graph()
    _ = gc.collect()

In [61]:
state = reset()
index = choose_action(0.88, state)
action,_ = action_for_each(index,k)
print(action , state)

[0, 0, 0] [0, 0, 0, 0, 0, 0]


In [62]:
import tensorflow as tf
import gc
import math

In [63]:
model = build_model(2*k,2**k)
scores = []
# Train the Q-Network
epsilon = 1
#epsilon_min = 0.01  # 0.0001 0.001
for episode in range(100):
    state = reset()
    rewards = []
    for j in range(int(running_time//time_interval)):
    
        # Select an action using epsilon-greedy strategy
        index = choose_action(epsilon, state)
        action,_ = action_for_each(index,k)
        # Take a step in the environment
        next_state, reward = env(action,state,3)
        rewards.append(reward)
        # Update the replay memory
        memory = update_memory(state, action, reward, next_state)

        state = next_state
        print(j)
        if len(memory) > batch_size:
            replay(memory,batch_size,state,epsilon)
            
        reset_tensorflow_keras_backend()
        
    scores.append(sum(rewards))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
0
1
2
3
4
5
6


KeyboardInterrupt: 

In [ ]:
# Plot the policy performance
fig = plt.figure()
ax = fig.add_subplot(111)
x = np.arange(1, len(scores) + 1)
y = scores
plt.scatter(x, y, marker='x', c=y)
fit = np.polyfit(x, y, deg=4)
p = np.poly1d(fit) 
plt.plot(x,p(x),"r--") 
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.title(' performance for tire')
plt.show()
     